Network architecture
In GoogleNet starts with two Conv-MaxPool blocks and then continues with a series of Inception blocks separated by Max Pool layers before the fineal Fully Connected layer.

Inception block
The Inception block is depicted at [iii].

It takes as input a tensor and passes it through 4 different streams:

a 1x1 Conv layer
a 1x1 Conv layer followed by a 3x3 Conv layer
a 1x1 Conv layer followed by a 5x5 Conv layer
a 3x3 Max Pool layer followed by a 1x1 Conv layer
The output tensors of all four final Conv layers are concatenated to one tensor.

In [2]:
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, \
     Concatenate, AvgPool2D, Dropout, Flatten, Dense
import numpy as np
from PIL import Image
import os
import matplotlib.pyplot as plt
import random
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import  Concatenate
from tensorflow.keras.layers import  Input
from tensorflow.keras.layers import  AvgPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.applications.vgg16 import preprocess_input


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
image_generator = ImageDataGenerator(rescale=1./255)
train_loader = image_generator.flow_from_directory(
    directory=r"/content/drive/MyDrive/Colab Notebooks/Data/train",
    target_size=(224, 224),
    batch_size=50,
    shuffle=True
)
test_loader = image_generator.flow_from_directory(
    directory=r"/content/drive/MyDrive/Colab Notebooks/Data/test",
    target_size=(224, 224),
    batch_size=50,
    shuffle=True
)
valid_loader = image_generator.flow_from_directory(
    directory=r"/content/drive/MyDrive/Colab Notebooks/Data/valid",
    target_size=(224, 224),
    batch_size=50,
    shuffle=True
)

Found 613 images belonging to 4 classes.
Found 315 images belonging to 4 classes.
Found 72 images belonging to 4 classes.


In [5]:
idx_to_class = {val:key for key, val in dict(train_loader.class_indices).items()}
idx_to_class

{0: 'adenocarcinoma',
 1: 'largecellcarcinoma',
 2: 'normal',
 3: 'squamouscellcarcinoma'}

model building and training

In [6]:


def inception_block(x, filters):
    t1 = Conv2D(filters=filters[0], kernel_size=1, activation='relu')(x)

    t2 = Conv2D(filters=filters[1], kernel_size=1, activation='relu')(x)
    t2 = Conv2D(filters=filters[2], kernel_size=3, padding='same', activation='relu')(t2)

    t3 = Conv2D(filters=filters[3], kernel_size=1, activation='relu')(x)
    t3 = Conv2D(filters=filters[4], kernel_size=5, padding='same', activation='relu')(t3)

    t4 = MaxPool2D(pool_size=3, strides=1, padding='same')(x)
    t4 = Conv2D(filters=filters[5], kernel_size=1, activation='relu')(t4)

    output = Concatenate()([t1, t2, t3, t4])
    return output


input = Input(shape=(224, 224, 3))
x = Conv2D(filters=64, kernel_size=7, strides=2, padding='same', activation='relu')(input)
x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)

x = Conv2D(filters=64, kernel_size=1, activation='relu')(x)
x = Conv2D(filters=192, kernel_size=3, padding='same', activation='relu')(x)
x = MaxPool2D(pool_size=3, strides=2)(x)

x = inception_block(x, filters=[64, 96, 128, 16, 32, 32])
x = inception_block(x, filters=[128, 128, 192, 32, 96, 64])
x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)

x = inception_block(x, filters=[192, 96, 208, 16, 48, 64])
x = inception_block(x, filters=[160, 112, 224, 24, 64, 64])
x = inception_block(x, filters=[128, 128, 256, 24, 64, 64])
x = inception_block(x, filters=[112, 144, 288, 32, 64, 64])
x = inception_block(x, filters=[256, 160, 320, 32, 128, 128])
x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)

x = inception_block(x, filters=[256, 160, 320, 32, 128, 128])
x = inception_block(x, filters=[384, 192, 384, 48, 128, 128])
x = AvgPool2D(pool_size=7, strides=1)(x)
x = Dropout(rate=0.4)(x)

x = Flatten()(x)
output = Dense(units=4, activation='softmax')(x)



model = Model(inputs=input, outputs=output)
model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

model.save("lungcancer2002.h5")

model_train_history = model.fit(
    x= train_loader,
    steps_per_epoch = 9,
    epochs = 200,
    validation_data = test_loader,
    validation_steps = 7
)


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/200
9/9 [==============================] - 248s 27s/step - loss: 1.4393 - accuracy: 0.2956 - val_loss: 1.3529 - val_accuracy: 0.3810
Epoch 2/200
9/9 [==============================] - 13s 2s/step - loss: 1.3739 - accuracy: 0.3089 - val_loss: 1.3430 - val_accuracy: 0.5397
Epoch 3/200
9/9 [==============================] - 9s 1s/step - loss: 1.3232 - accuracy: 0.3711 - val_loss: 1.3129 - val_accuracy: 0.3810
Epoch 4/200
9/9 [==============================] - 11s 1s/step - loss: 1.2934 - accuracy: 0.3390 - val_loss: 1.2170 - val_accuracy: 0.4190
Epoch 5/200
9/9 [==============================] - 9s 1s/step - loss: 1.1417 - accuracy: 0.4649 - val_loss: 0.9582 - val_accuracy: 0.5492
Epoch 6/200
9/9 [==============================] - 8s 941ms/step - loss: 1.0197 - accuracy: 0.4939 - val_loss: 0.9213 - val_accuracy: 0.5524
Epoch 7/200
9/9 [==============================] - 7s 766ms/step - loss: 0.9322 - accuracy: 0.5327 - val_loss: 0.8954 - val_accuracy: 0.5524
Epoch 8/200
9/9 [======